# **INTRODUCTIONS**

---

The code is separated into multiple sections:

1. **[Tables](#tables)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb",
        "src/statics/reclassified_municipality_mapping.ipynb",
        "src/statics/populations/cbs_population_rwzi_mutations.ipynb"
    ]
}
```

# **TABLES**

---

### STAGES

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE STAGE SEQUENCE(S).....
IF NOT EXISTS(SELECT * FROM SYS.SEQUENCES WHERE OBJECT_ID = OBJECT_ID(N'[DBO].[SEQ_VWSSTAGE_CBS_POPULATION_RWZI]') AND TYPE = 'SO')
CREATE SEQUENCE [SEQ_VWSSTAGE_CBS_POPULATION_RWZI]
  START WITH 1
  INCREMENT BY 1;
GO

-- 2) CREATE STAGE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTAGE].[CBS_POPULATION_RWZI]'))
CREATE TABLE [VWSSTAGE].[CBS_POPULATION_RWZI](
  [ID] INT PRIMARY KEY NOT NULL DEFAULT (NEXT VALUE FOR [dbo].[SEQ_VWSSTAGE_CBS_POPULATION_RWZI]),
  [RWZI_CODE] VARCHAR(100) NULL,
  [RWZI_NAAM] VARCHAR(100) NULL,
  [STARTDATUM] VARCHAR(100) NULL,
  [EINDDATUM] VARCHAR(100) NULL,
  [INWONERS] VARCHAR(100) NULL,
  [REGIO_TYPE] VARCHAR(100) NULL,
  [REGIO_CODE] VARCHAR(100) NULL,
  [REGIO_NAAM] VARCHAR(100) NULL,
  [AANDEEL] VARCHAR(100) NULL,
  [TOELICHTING] VARCHAR(100) NULL,
  [DATE_LAST_INSERTED]  DATETIME DEFAULT GETDATE()
);
GO


### STATICS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE STAGE SEQUENCE(S).....
IF NOT EXISTS(SELECT * FROM SYS.SEQUENCES WHERE [OBJECT_ID] = OBJECT_ID(N'[DBO].[SEQ_VWSSTATIC_CBS_POPULATION_RWZI]') AND [TYPE] = 'SO')
CREATE SEQUENCE [SEQ_VWSSTATIC_CBS_POPULATION_RWZI]
  START WITH 1
  INCREMENT BY 1;
GO

-- 2) CREATE STAGE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTATIC].[CBS_POPULATION_RWZI]'))
CREATE TABLE [VWSSTATIC].[CBS_POPULATION_RWZI] (
  [ID] INT PRIMARY KEY NOT NULL DEFAULT (NEXT VALUE FOR [dbo].[SEQ_VWSSTATIC_CBS_POPULATION_RWZI]),
  [RWZI_CODE] VARCHAR(100) NULL,
  [RWZI_NAAM] VARCHAR(100) NULL,
  [STARTDATUM] DATE NULL,
  [EINDDATUM] DATE NULL,
  [INWONERS] INT NULL,
  [REGIO_TYPE] VARCHAR(100) NULL,
  [REGIO_CODE] VARCHAR(100) NULL,
  [REGIO_NAAM] VARCHAR(100) NULL,
  [AANDEEL] decimal(19,3) NULL,
  [DATE_LAST_INSERTED]  DATETIME DEFAULT GETDATE()
);
GO

# **STORED PROCEDURES**

---

In de stored procedure **SP\_MERGE\_MOVE\_STATIC\_CBS\_RWZI** worden wijzigingen verwerkt in de tabel **VWSStatic.cbs\_population\_rwzi.** Dit is bedoeld om wijzigingen in rwzi's gedurende een lopend jaar aan te brengen. De procedure eindigt met een mapping tussen rwzi's en gemeenten en veiligheidsregio's. Onderin de procedure is een wijziging aangebracht, het 'oude' resultaat is verhuisd naar de common-table-expression 'Selectect\_CTE' en deze wordt in het uiteindelijke Select statement geaggregeerd naar de gewenste gemeentelijke indeling m.b.v. de mapping tabel **VWSStatic.Recode\_Municipality** (die ook wordt gebruikt bij de aanpassingen van de Population\_Base). 

N.B. in de mapping-tabel moet dan wel de kolom Active op 1 staan om te worden uitgevoerd.

### STAGING &rarr; STATIC MAPPING

In [ ]:
-- COPYRIGHT (C) 2022 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

CREATE OR ALTER   PROCEDURE [DBO].[SP_MERGE_MOVE_STATIC_CBS_RWZI]
AS
BEGIN
    WITH COMBINED_CBS_RWZI
    AS (
        SELECT 
            CAST([RWZI_CODE] AS INT) AS [RWZI_CODE]
            ,[RWZI_NAAM]
            ,CONVERT(DATE,[STARTDATUM],105) AS [STARTDATUM]
            ,CASE 
                WHEN LEN([EINDDATUM]) = 0 THEN CAST('9999-12-31' AS DATE) 
                ELSE CONVERT(DATE,[EINDDATUM],105) 
            END AS [EINDDATUM]
            ,[INWONERS]
            ,[REGIO_TYPE]
            ,[REGIO_CODE]
            ,[REGIO_NAAM]
            ,[AANDEEL]  
            , 'base' AS [TYPE]
        FROM [VWSSTAGE].[CBS_POPULATION_RWZI]
        WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSSTAGE.CBS_POPULATION_RWZI) AND LOWER([TOELICHTING]) = 'definitief'
        UNION
        SELECT 
            CAST([RWZI_CODE] AS INT)                AS [RWZI_CODE]
            ,[RWZI_NAAM]
            ,CONVERT(DATE,[STARTDATUM],105) AS [STARTDATUM]
            ,CASE WHEN LEN([EINDDATUM]) = 0 THEN CAST('9999-12-31' AS DATE) ELSE
            
                CONVERT(DATE,[EINDDATUM],105) 
                END
            ,[INWONERS]
            ,[REGIO_TYPE]
            ,[REGIO_CODE]
            ,[REGIO_NAAM]
            ,[AANDEEL]    
            ,'mutation' AS [TYPE]
        FROM [VWSSTAGE].[CBS_POPULATION_RWZI_MUTATIONS]
        WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSSTAGE.CBS_POPULATION_RWZI_MUTATIONS)
    ),
SELECTED_CTE AS (
    SELECT	 [RWZI_CODE]
			,[RWZI_NAAM]
			,CASE
				-- FOR THE RWZI / REGION COMBINATIONS THAT START AT 1-1-2021, CHANGE START DATE TO 1-1-2020
				WHEN
					-- ONLY SELECT RWZI / REGION COMBINATIONS THAT START AT 1-1-2021
					MIN(STARTDATUM) OVER (PARTITION BY RWZI_CODE,REGIO_CODE) = CAST('2021-01-01' AS DATE)
					-- FOR THE RWZI / REGION COMBINATION WHERE THE DATE IS ADJUSTED, ONLY ADJUST DATE 1-1-2021 TO 1-1-2020
					AND MIN(STARTDATUM) OVER (PARTITION BY RWZI_CODE,REGIO_CODE, STARTDATUM) = CAST('2021-01-01' AS DATE)
				THEN CAST('2020-01-01' AS DATE)
				ELSE [STARTDATUM]
			END                         AS [STARTDATUM]
			,ISNULL([EINDDATUM], CAST('9999-12-31' AS DATE)) AS [EINDDATUM]
			,[INWONERS]
			,[REGIO_TYPE]
			,[REGIO_CODE]
			,[REGIO_NAAM]
			,PARSE(REPLACE([AANDEEL], '%', '') AS DECIMAL(19,3) USING 'nl-NL') AS [AANDEEL]
			--,[TYPE]
    FROM 
        COMBINED_CBS_RWZI
    WHERE 
        RWZI_CODE > 0

)
    INSERT INTO [VWSSTATIC].CBS_POPULATION_RWZI (
        [RWZI_CODE]
        ,[RWZI_NAAM]
        ,[STARTDATUM]
        ,[EINDDATUM]
        ,[INWONERS]
        ,[REGIO_TYPE]
        ,[REGIO_CODE]
        ,[REGIO_NAAM]
        ,[AANDEEL]
      )
    SELECT 
         A.[RWZI_CODE]
        ,A.[RWZI_NAAM]
        ,A.[STARTDATUM]
        ,A.[EINDDATUM]
        ,MAX(A.[INWONERS]) AS INWONERS
        ,A.[REGIO_TYPE]
        ,ISNULL(B.[GM_CODE_NEW],A.[REGIO_CODE]) AS REGIO_CODE
        ,ISNULL(B.[GM_CODE_NEW],A.[REGIO_NAAM]) AS REGIO_NAAM
        ,SUM((A.[AANDEEL]/100) * ISNULL(B.[SHARES]/100,1.0)) AS AANDEEL
        --,[TYPE]
    FROM SELECTED_CTE A
		    LEFT JOIN   (
                    SELECT 
                        * 
                    FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING] 
                    WHERE [ACTIVE]=1
                        AND [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING])  
                ) B ON A.[REGIO_CODE] = B.[GM_CODE_ORIGINAL]
	GROUP BY A.[RWZI_CODE]
			,A.[RWZI_NAAM]
			,A.[STARTDATUM]
			,A.[EINDDATUM]
			,A.[REGIO_TYPE]
			,ISNULL(B.[GM_CODE_NEW],A.[REGIO_CODE]) 
			,ISNULL(B.[GM_CODE_NEW],A.[REGIO_NAAM]) 
END
GO

In [ ]:
EXEC [DBO].[SP_MERGE_MOVE_STATIC_CBS_RWZI]
GO

# **DATATINO CONFIGURATIONS**

---


### WORKFLOWS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) UPSERT WORKFLOW(S).....
DECLARE @workflow_name NVARCHAR(50) = 'STATIC_DATA',
        @workflow_id INT,        
        @workflow_description VARCHAR(256),
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

-- 2) UPSERT WORKFLOW(S).....
SELECT TOP(1)
    @workflow_id = workflows.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] AND v_workflows.[ID] = workflows.[ID]
WHERE v_workflows.[NAME] = @workflow_name;

SET @workflow_description = CONCAT('WORKFLOW: ', @workflow_name);

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_WORKFLOW]
    @id = @workflow_id, 
    @workflow_name = @workflow_name,
    @description = @workflow_description,
    @SCHEDULE = '0 12 * */3 *', -- AT 12:00 PM, EVERY 3 MONTHS
    @active = @is_active;

-- 2) UPSERT SOURCE(S).....
DECLARE @source NVARCHAR(256),
        @source_id INT,
        @source_name NVARCHAR(256),
        @source_description NVARCHAR(256),
        @target_name VARCHAR(256),
        @location_type VARCHAR(50),
        @security_profile VARCHAR(50);

-- 2.1) SET ENVIRONMENTAL VARIABLES.....
SET @source = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'datafiles/STATIC/Aantal_inwoners_per_verzorgingsgebied.csv'
    WHEN 'acceptance' THEN 'datafiles/STATIC/Aantal_inwoners_per_verzorgingsgebied.csv'
    ELSE 'datafiles/STATIC/Aantal_inwoners_per_verzorgingsgebied.csv'
END;

SET @location_type = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'AzureBlob'
    WHEN 'acceptance' THEN 'AzureBlob'
    ELSE 'AzureBlob'
END;

SET @security_profile = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'RIVM'
    WHEN 'acceptance' THEN 'RIVM'
    ELSE 'RIVM'
END;

-- 2.2) UPSERT TABLE SOURCE(S): STAGING......
SET @source_name = 'SOURCE_Load_Inwoners_per_verzorgingsgebied';
SET @source_description = CONCAT('LOAD: CBS_POPULATION_RWZI - ', @workflow_name);
SET @target_name = 'VWSSTAGE.CBS_POPULATION_RWZI';

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = 'rwzi_code|rwzi_naam|startdatum|einddatum|inwoners|regio_type|regio_code|regio_naam|aandeel|toelichting',
    @target_columns = 'RWZI_CODE|RWZI_NAAM|STARTDATUM|EINDDATUM|INWONERS|REGIO_TYPE|REGIO_CODE|REGIO_NAAM|AANDEEL|TOELICHTING',
    @target_name = @target_name,
    @source_type = 'CsvFile',
    @location_type = @location_type,
    @delimiter_type = 'Colon',
    @security_profile= @security_profile;

-- 2.3) UPSERT STORED PROCEDURE SOURCE(S): STAGE TO STATIC......
SET @source = 'DBO.SP_MERGE_MOVE_STATIC_CBS_RWZI';
SET @source_name = 'SOURCE_SP_MERGE_MOVE_STATIC_CBS_RWZI';
SET @source_description = 'MAP: CBS_POPULATION_RWZI FROM STAGE TO STATIC';

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = null,
    @target_columns = null,
    @target_name = null,
    @source_type = 'StoredProcedure',
    @location_type = 'N/A',
    @delimiter_type = 'N/A',
    @security_profile= @security_profile;

-- 3) UPSERT PROCESS(ES).....
DECLARE @process_id INT,
        @process_name NVARCHAR(256),
        @process_description NVARCHAR(256),
        @process_source_name VARCHAR(256);

-- 3.1) UPSERT TABLE PROCESS(ES): STAGING......
SET @process_name = 'PROCESS_Load_Inwoners_per_verzorgingsgebied';
SET @process_description = CONCAT('LOAD: CBS_POPULATION_RWZI - ', @workflow_name);
SET @process_source_name = 'SOURCE_Load_Inwoners_per_verzorgingsgebied';

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;

-- 3.2) UPSERT STORED PROCEDURED PROCESS(S): STAGE TO STATIC......
SET @process_name = 'PROCESS_SOURCE_SP_MERGE_MOVE_STATIC_CBS_RWZI';
SET @process_description = 'MAP: CBS_POPULATION_RWZI FROM STAGE TO STATIC';
SET @process_source_name = 'SOURCE_SP_MERGE_MOVE_STATIC_CBS_RWZI';

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;

-- 4) UPSERT DEPENDENC(Y)(IES).....
DECLARE @dependency_id INT,
        @dependency_name NVARCHAR(256),
        @dependency_description NVARCHAR(256),
        @dependency_dataflow_name NVARCHAR(256),
        @dependency_process_name NVARCHAR(256);

-- 4.1) UPSERT STAGING TO STORED PROCEDURE DEPENDENC(Y)(IES).....
SET @dependency_name = 'DEPENDENCY_PROCESS_SOURCE_SP_MERGE_MOVE_STATIC_CBS_RWZI_MUTATIONS';
SET @dependency_dataflow_name = 'PROCESS_SOURCE_SP_MERGE_MOVE_STATIC_CBS_RWZI';
SET @dependency_process_name = 'PROCESS_Load_Inwoners_per_verzorgingsgebied_mutaties';
SET @dependency_description = CONCAT('TRIGGER ', @dependency_dataflow_name,' AFTER ', @dependency_process_name ,' HAS FINISHED');

SELECT TOP(1)
    @dependency_id = dependencies.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
WHERE dependencies.[NAME] = @dependency_name
    AND v_dependencies.[WORKFLOW_NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
    @id = @dependency_id,
    @dataflow_name = @dependency_dataflow_name,
    @dataflowtype_id = 2,
    @dependency_name = @dependency_process_name,
    @dependencytype_id = 2,
    @workflow_name = @workflow_name,
    @name = @dependency_name,
    @description = @dependency_description,
    @active = 1;
GO

-- 4.2) UPSERT STAGING TO STORED PROCEDURE DEPENDENC(Y)(IES).....
SET @dependency_name = 'DEPENDENCY_PROCESS_SOURCE_SP_MERGE_MOVE_STATIC_CBS_RWZI';
SET @dependency_dataflow_name = 'PROCESS_SOURCE_SP_MERGE_MOVE_STATIC_CBS_RWZI';
SET @dependency_process_name = 'PROCESS_Load_Inwoners_per_verzorgingsgebied';
SET @dependency_description = CONCAT('TRIGGER ', @dependency_dataflow_name,' AFTER ', @dependency_process_name ,' HAS FINISHED');

SELECT TOP(1)
    @dependency_id = dependencies.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
WHERE dependencies.[NAME] = @dependency_name
    AND v_dependencies.[WORKFLOW_NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
    @id = @dependency_id,
    @dataflow_name = @dependency_dataflow_name,
    @dataflowtype_id = 2,
    @dependency_name = @dependency_process_name,
    @dependencytype_id = 2,
    @workflow_name = @workflow_name,
    @name = @dependency_name,
    @description = @dependency_description,
    @active = 1;
GO